In [1]:
# conda list
import glob

In [2]:
from google.cloud import storage, bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])

In [3]:
storage_client = storage.Client(credentials=credentials, project=credentials.project_id)
ssd_bucket = 'scanny-data-backup'
bucket = storage_client.get_bucket(ssd_bucket)

In [4]:
for obj in bucket.list_blobs(prefix=None):
    print(obj.name)

Clemow Data/2019 Box Count W49-W13.xlsx
Clemow Data/2019 Recipe Detail W49 - W13.xlsx
Clemow Data/2019 Scanny Data W49-W13.xlsx
W06/Kitting Dashboard -- W06.pdf
W06/W06.xlsx
W06/Week06Kitting.pdf
W06/prepared.csv
W06/raw.xlsx
W06/staged.csv
W07/Kitting Dasboard -- W07.pdf
W07/W07.xlsx
W07/W07kitting.pdf
W07/prepared.csv
W07/raw.xlsx
W07/staged.csv
W08/Kitting KPI Dashboard - W08.pdf
W08/W08 Assembly.xlsx
W08/W08.xlsx
W08/prepared.csv
W08/raw.xlsx
W08/staged.csv
W09/W09 Assembly.xlsx
W09/W09.xlsx
W09/W09Kitting KPIs.pdf
W09/prepared.csv
W09/raw.xlsx
W09/staged.csv
W10/Assembly -- W10.xlsx
W10/Kitting KPI Dashboard - W10.pdf
W10/prepared.csv
W10/raw.xlsx
W10/staged.csv
W11/Kitting KPI Dashboard--W11.pdf
W11/W11-Assembly.xlsx
W11/prepared.csv
W11/raw.xlsx
W11/staged.csv
W12/Kitting KPI Dashboard -- W12.pdf
W12/W12-Assembly.xlsx
W12/prepared.csv
W12/raw.xlsx
W12/staged_12.csv
W13/Kitting KPI Dashboard - W13.pdf
W14/Kitting KPI Dashboard - W14.pdf
W14/prepared.csv
W14/raw.xlsx
W14/staged.cs

In [5]:
def download_from_gcs(gcs_folder, local_path):
    """
    gcs_folder -- the blob storage of the weeks' data
    local_path -- point to the desginated local folder 
    """
    for obj in bucket.list_blobs(prefix = gcs_folder):
        fileName = obj.name.split('/')[-1]
        obj.download_to_filename(os.path.join(local_path,fileName))

In [6]:
cusomized_schema =[
        bigquery.SchemaField("Start_Time", "TIMESTAMP"),
        bigquery.SchemaField("Finish_Time", "TIMESTAMP"),
        bigquery.SchemaField("Activity", "STRING"),
        bigquery.SchemaField("Seq_Code", "STRING"),
        bigquery.SchemaField("Recipe_Name", "STRING"),
        bigquery.SchemaField("Break_Reasons", "STRING"),
        bigquery.SchemaField("Missing_Ingredients", "STRING"),
        bigquery.SchemaField("Kitting_Line", "STRING"),
        bigquery.SchemaField("Assembly_Batch", "STRING"),
        bigquery.SchemaField("Event_Shift", "STRING"),
        bigquery.SchemaField("Team_Leader", "STRING"),
        bigquery.SchemaField("Pickers_Count", "INTEGER"),
        bigquery.SchemaField("Time_Consumption", "FLOAT"),
        bigquery.SchemaField("Week", "STRING"),
    ]

In [7]:
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)
datasets = list(bigquery_client.list_datasets())
for dataset in datasets:
    print(dataset.dataset_id)

PowerBI
scanning_data_backup
test_data_set


In [8]:
type(bigquery_client.get_table('scanning-database.scanning_data_backup.W29')) #table reference

google.cloud.bigquery.table.Table

In [9]:
def create_table(dataset_id, gcs_uri, new_table_id):
    dataset_ref = bigquery_client.dataset(dataset_id) 
    job_config = bigquery.LoadJobConfig()
    ##new way to configure job's arrtibutes
    job_config.autodetect = True ## auto detect, default is False
    # job_config.schema = cusomized_schema ## choose whether to use customized schema
    # job_config.skip_leading_rows = 1 ## if auto detect then eliminate this
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.source_format = bigquery.SourceFormat.CSV
    load_job = bigquery_client.load_table_from_uri(gcs_uri, dataset_ref.table(new_table_id), job_config=job_config)
    print("Starting job {}".format(load_job.job_id))
    load_job.result()  # Waits for table load to complete.
    print("Job finished.")
    destination_table = bigquery_client.get_table(dataset_ref.table(new_table_id))
    print("Loaded {} rows.".format(destination_table.num_rows))

In [10]:
"""
Change the directory week-by-week
"""
uri = "gs://scanny-data-backup/W44/prepared.csv" ## <= 1/2
datasetID = "scanning_data_backup"
new_table_name = "W44" ## <= 2/2

In [11]:
create_table(datasetID, uri, new_table_name)

Starting job 3413e33f-a553-4133-8ff8-e7478b6548ea
Job finished.
Loaded 7699 rows.
